# Notebook to transform labels in a Yolo configuration


## Import useful modules


In [ ]:
# General modules
import os
import subprocess
import shutil
import random
import time

# Data manipulation modules
import pandas as pd
import json
from sklearn.model_selection import train_test_split

# Image and video modules
from PIL import Image
from plotly import express as px
import cv2

# Machine learning module
from ultralytics import YOLO

## Launch `label-studio` to label images


In [ ]:
os.environ["LABEL_STUDIO_LOCAL_FILES_SERVING_ENABLED"] = "true"
os.environ["LABEL_STUDIO_LOCAL_FILES_DOCUMENT_ROOT"] = os.getcwd()

subprocess.run("label-studio")

## Read `label-studio` labels file and available classes


In [2]:
with open("./assets/labels.json", encoding="utf-8") as f:
    annotations = json.load(f)

classes = pd.read_csv("./assets/classes.csv", names=["_"])["_"].to_list()

## Display dataset images


In [3]:
def generate_image(annotation):
    path = "/".join(annotation["image"].split("/")[-2:])
    fig = px.imshow(Image.open(path))
    for label in annotation.get("label", []):
        x = round(label["x"] / 100 * label["original_width"])
        y = round(label["y"] / 100 * label["original_height"])
        width = round(label["width"] / 100 * label["original_width"])
        height = round(label["height"] / 100 * label["original_height"])
        str_label = label["rectanglelabels"][0]
        fig.add_shape(
            type="rect",
            x0=x,
            x1=x + width,
            y0=y,
            y1=y + height,
            xref="x",
            yref="y",
            line_color="red",
        )
        fig.add_annotation(
            x=x,
            y=y,
            text=str_label,
            xref="x",
            xanchor="left",
            yref="y",
            yanchor="bottom",
            showarrow=False,
            font_color="white",
            bgcolor="red",
        )
    return fig

_Run the next cell to explore randomly your dataset_


In [131]:
fig = generate_image(random.choice(annotations))

fig.update_layout(margin=dict(l=20, r=20, t=20, b=20), width=640, height=500)

## Check the label distribution


In [38]:
count = {}
for annotation in annotations:
    labels = annotation.get("label", [])
    if len(labels):
        for l in labels:
            k = l["rectanglelabels"][0]
            count[k] = count.get(k, 0) + 1
    else:
        count["background"] = count.get("background", 0) + 1

In [39]:
classes_plus = classes + ["background"]
fig = px.bar(
    x=classes_plus,
    y=[count[c] for c in classes_plus],
    color=[c.split(" ")[0] for c in classes_plus],
)
fig.update_layout(
    xaxis={"dtick": classes_plus},
    xaxis_title=None,
    yaxis_title="Annotation count",
    legend_title="Suit",
)

## Generate Yolo annotation file


In [132]:
def generate_yolo_annotations(annotations):
    for a in annotations:
        path = a["image"].split("/")[-1]
        with open(f"labels/{path[:-3]}txt", mode="w", encoding="utf8") as f:
            for l in a.get("label", []):
                x_min = l["x"] / 100
                y_min = l["y"] / 100
                box_width = l["width"] / 100
                box_height = l["height"] / 100
                x_center = x_min + box_width / 2
                y_center = y_min + box_height / 2
                label = classes.index(l["rectanglelabels"][0])
                f.write(
                    f"{label} {x_center:.3f} {y_center:.3f} {box_width:.3f} {box_height:.3f}\n"
                )

In [133]:
!mkdir labels

In [134]:
generate_yolo_annotations(annotations)

## Split dataset into train, valid and test dataset


In [135]:
# Read images and annotations
images = [
    os.path.join("dataset", x)
    for x in os.listdir("dataset")
    if x[-3:] == "jpg"
    if x[:-3] in [y[:-3] for y in os.listdir("labels")]
]
labels = [os.path.join("labels", x) for x in os.listdir("labels") if x[-3:] == "txt"]

images.sort()
labels.sort()

# Split the dataset into train-valid-test splits
train_images, val_images, train_labels, val_labels = train_test_split(
    images, labels, test_size=0.2, random_state=1
)
val_images, test_images, val_labels, test_labels = train_test_split(
    val_images, val_labels, test_size=0.5, random_state=1
)

### Create directories to move images and labels


In [136]:
!mkdir images images/train images/val images/test labels/train labels/val labels/test

### Move files in directories


In [137]:
# Utility function to move images
def move_files_to_folder(list_of_files, destination_folder, move=False):
    for f in list_of_files:
        try:
            if move:
                shutil.move(f, destination_folder)
            else:
                shutil.copy(f, destination_folder)
        except:
            print(f)
            assert False


# Copy the images into their folders
move_files_to_folder(train_images, "images/train")
move_files_to_folder(val_images, "images/val/")
move_files_to_folder(test_images, "images/test/")
# Move the labels into their folders
move_files_to_folder(train_labels, "labels/train/", move=True)
move_files_to_folder(val_labels, "labels/val/", move=True)
move_files_to_folder(test_labels, "labels/test/", move=True)

# Train


In [ ]:
os.environ["WANDB_DISABLED"] = "true"
model = YOLO("yolov8n.pt")

model.train(
    data="tarot_card_data.yaml", epochs=400, imgsz=640, device="mps", resume=True
)

## Valid


In [ ]:
model = YOLO("./runs/detect/train7/weights/last.pt")

# Customize validation settings
validation_results = model.val(
    data="tarot_card_data.yaml", imgsz=640, batch=16, conf=0.5, iou=0.7, device="mps"
)

## Track


In [ ]:
model = YOLO("./runs/detect/train7/weights/last.pt")

# Open the video file
video_path = "http://192.168.1.10:4747/video"
cap = cv2.VideoCapture(video_path)
fps = 20
start = time.time()
# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()
    now = time.time()
    if success and ((now - start) > 1 / fps):
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        start = time.time()
        results = model.track(frame, stream=True, persist=True)

        # Visualize the results on the frame
        results = [result for result in results]
        annotated_frame = results[0].plot()

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

## Convert to tensorflow light app


In [5]:
model = YOLO("./runs/detect/train7/weights/last.pt")

In [ ]:
model_onnx = model.export(format="onnx")

import onnx2tf

tf_model = onnx2tf.convert(model_onnx, output_tfv1_pb=True)